In [6]:
import numpy as np
import sys
# sys.path.insert(0, '/Users/pouyabashivan/Dropbox (MIT)/Codes/dldata')
import dldata.metrics.utils as utils
import dldata.stimulus_sets.hvm as hvm
import cPickle
import h5py
import sys
import os
from sklearn.cross_validation import KFold
from sklearn import cross_decomposition
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
np.random.seed([0])
from dldata.physiology.hongmajaj.mappings import CHANNEL_INFO
from sklearn.preprocessing import scale
%matplotlib inline

import seaborn as sns
from scipy.misc import imread
from scipy.stats import pearsonr
np.random.seed(123)
import tensorflow.contrib.slim as slim

%matplotlib inline
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops       
import scipy.stats as stats

sys.path.insert(0, '/braintree/home/bashivan/dropbox/Codes/Retina/misc/Models/')
from metrics_expert import MetricsExpert
analyst = MetricsExpert('/braintree/data2/active/users/bashivan/model_features/alexnet_model_139k.hdf5',
                        '/braintree/data2/active/users/bashivan/tmp',
                        'IT')

sns.set_style("white")
sns.set_style("ticks", {"xtick.major.size": 14, "ytick.major.size": 14})
sns.set_context("poster")


dataset_hvm = hvm.HvMWithDiscfade()
meta_hvm = dataset_hvm.meta
neural_fea = np.array(dataset_hvm.neuronal_features)
neural_reps = dataset_hvm.get_neuronal_feature_reps()['V6']

common attributes, forcing a renaming ...
Using default renamer ...
('Replacing columns', ['rxz', 'rxy', 'ryz'])
common attributes, forcing a renaming ...
Using default renamer ...
('Replacing columns', ['rxz', 'rxy', 'ryz'])


# Utility Functions

In [ ]:
from sklearn.linear_model import LinearRegression
from scipy.interpolate import griddata, interp2d


def reps_to_array(reps):
    """
    reps: dictionary containing the reps for each variation level  
    """
    max_reps = np.max([reps[i].shape[0] for i in reps.keys()], axis=0)
    hvm_neural = np.zeros((max_reps, 5760, reps['V0'].shape[2]))
    hvm_neural[...] = np.NaN

    c = 0
    for key in reps:
        shape = reps[key].shape
        hvm_neural[:shape[0], c:c+shape[1], :] = reps[key]
        c += shape[1]
    return hvm_neural


def concat_reps(rep_list):
    """
    """
    max_reps = np.max([r.shape[0] for r in rep_list])
    resized_reps = []
    for r in rep_list:
        tmp = np.zeros((max_reps, r.shape[1], r.shape[2]))
        tmp[...] = np.NaN
        tmp[:r.shape[0], :, :] = r
        resized_reps.append(tmp)
    return np.concatenate(resized_reps, axis=-1)

def fix_nan_reps(reps):
    """Some of the entries in neural reps might be nan. 
    Substitute those values by the average response of 
    corresponding neurons to all images over all valid reps.
    reps = [n_reps, n_samples, n_neurons]
    """
    if np.any(np.isnan(reps)):
        # find the indices of nan neurons
        nan_ind = np.isnan(reps)
        _, _, nan_neu_ind = np.nonzero(nan_ind)

        corrected_reps = reps
        for n in np.unique(nan_neu_ind):
            # create a mask of all nan values for a neuron
            mask = np.zeros(shape=nan_ind.shape, dtype=bool)
            mask[:, :, n] = True
            masked_nan_ind = nan_ind & mask

            # substitue all nan values of neuron by average neuron response
            av_neuron_act = np.nanmean(reps[:, :, n])
            corrected_reps[masked_nan_ind] = av_neuron_act
        return corrected_reps
    else:
        return reps

def project_reps(input_reps, W_mat):
    """Project each rep of neural data using the projection matrix
    input_reps = [n_reps, n_samples, n_neurons]"""
    input_reps = fix_nan_reps(input_reps)
    reps = []
    for rep in input_reps:
        reps.append(scale(rep))
    comp_reps = np.tensordot(reps, W_mat, axes=1)
    return comp_reps

def fit_reg(X, Y):
    """
    Fits a linear regression model to the data and returns regression model with score and predictions"""
    reg = LinearRegression()
    reg.fit(X, Y)
    preds = reg.predict(X)
    score = pearsonr(Y, preds)
    return reg, score, preds

def predict_outputs(features, weights):
    """
    Predict outputs given input features and weights."""
    model_pcs = np.matmul(features - weights['pca_b'], weights['pca_w'])
    preds = np.matmul(model_pcs, weights['pls_w']) + weights['pls_b'] 
    return preds


def resize_mat(mat, new_size):
    """
    Resize a matrix to the desired size. Input size is [num_channels, num_pixels, num_pixels]"""
    if mat.ndim==2:
        mat = np.expand_dims(mat, axis=0)
    num_ch, _, num_pix = np.array(mat).shape

    x = np.arange(0, num_pix)
    y = np.arange(0, num_pix)
    ratio = (new_size - 1.) / (num_pix - 1)

    x_new = np.arange(0, new_size)
    y_new = np.arange(0, new_size)

    output = []
    for i in range(num_ch):
        resized_rf_func = interp2d(x * ratio, y * ratio, mat[i], kind='cubic')
        tmp_out = resized_rf_func(x_new, y_new)
        output.append(tmp_out)

    return np.squeeze(output)    


# Separable (Mask-Mix) Mapping

In [12]:
# Load V6 ids
# ids = np.nonzero(meta_hvm['var'] == 'V6')[0]
# len(ids)

# Ko 640 ids
from scipy.io import loadmat

data = loadmat('/braintree/home/bashivan/dropbox/Data/dimensionality/Ko_data/V4_data/mag_left_v4.mat')
images_lookup = loadmat('/braintree/home/bashivan/dropbox/Data/dimensionality/Ko_data/id_object_lookup.mat')['id_object_lookup']
# site_locations = loadmat('/braintree/home/bashivan/dropbox/Data/Ko_data/V4_data/kk_2_pb_sites.mat')

# Find matching image ids on HVM
ids = []
for r in images_lookup:
    image_num = int(np.nonzero(meta_hvm['id'] == r[1])[0])
    ids.append(image_num)

# All ids
# ids = range(5760)


In [8]:
monkey='magneto'
v4_data_reps_hvm = h5py.File('/braintree/data2/active/common/for_pouya/season7/{0}/mapHVM_640/rate_img.mat'.format(monkey))
v4_data_reps_hvm = np.array(v4_data_reps_hvm['rate_img']).transpose(2, 0, 1)

v4_norm_reps_hvm = h5py.File('/braintree/data2/active/common/for_pouya/season7/{0}/mapHVM_640/rate_norm.mat'.format(monkey))
v4_norm_reps_hvm = np.array(v4_norm_reps_hvm['rate_norm']).transpose(2, 0, 1)

# Normalize responses    
bias = np.nanmean(v4_norm_reps_hvm, axis=0).mean(0)
std = np.nanmean(v4_norm_reps_hvm, axis=0).std(0)
neural_reps = (v4_data_reps_hvm - bias)/std
neural_fea = np.nanmean(neural_reps, axis=0)


In [18]:
len(ids)

640

In [16]:
from sklearn import linear_model

n_comps = 1000
area = 'it'  # v4 or it

# if area == 'v4':
#     neuron_ids = dataset_hvm.V4_NEURONS
#     layer_name = 'mdl_conv3'
# else:
#     neuron_ids = dataset_hvm.IT_NEURONS
#     layer_name = 'mdl_fc6'
neuron_ids = range(96)
    
features_h5_hvm = h5py.File('/braintree/data2/active/users/bashivan/model_features/alexnet_retina299_input299_out299.h5')
pca_neural = PCA(whiten=True, n_components=20)

# features = np.array(features_h5_hvm[layer_name])[ids, :]
# features = features.reshape(features.shape[0], -1)
# pca = PCA(n_components=n_comps, whiten=False)
# model_pcs = pca.fit_transform(features)

# neural_fr = neural_fea[ids][:, neuron_ids]
neural_fr = neural_fea

clf = linear_model.Ridge(alpha=10)

np.random.seed(0)
num_images = features.shape[0]
# train_ind = np.random.choice(range(num_images), num_images * 9 // 10, replace=False)
train_ind = np.random.choice(range(num_images), num_images // 2, replace=False)
test_ind = np.nonzero(~ np.in1d(range(num_images), train_ind))[0]

clf.fit(model_pcs[train_ind], neural_fr[train_ind])

# print the test performance 
hvm_preds = clf.predict(model_pcs[test_ind])
hvm_scores = np.array([pearsonr(hvm_preds[:, i], neural_fr[test_ind, i])[0] for i in range(hvm_preds.shape[-1])])

consistency = np.array(analyst.compute_consistency(neural_reps[:, :, neuron_ids], 20, population=False)).mean(1)
print('Scores: ', np.median(hvm_scores))
print('Consistency: ', np.median(consistency)) 
print('Norm. EV: ', np.median(hvm_scores ** 2 / consistency ** 2))

('Scores: ', 0.6797741056401212)
('Consistency: ', 0.8560523943000633)
('Norm. EV: ', 0.6575022540610389)


In [23]:
from sklearn.decomposition import PCA
import h5py
from scipy.stats import pearsonr

import sys
sys.path.insert(0, '/braintree/home/bashivan/dropbox/Codes/cube_mapping/cubemap/')
from sep_map import SeparableMap
# from cubemap.sep_map import SeparableMap

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

map_type = 'separable'   # 'linreg' or 'separable'
area = 'it'  # v4 or it

# if area == 'v4':
#     neuron_ids = dataset_hvm.V4_NEURONS
#     layer_name = 'mdl_conv3'
# else:
#     neuron_ids = dataset_hvm.IT_NEURONS
#     layer_name = 'mdl_pool5'
neuron_ids = range(96)

# features_h5_hvm = h5py.File('/braintree/data2/active/users/bashivan/model_features/alexnet_retina299_input299_out299.h5')
# feats = np.array(features_h5_hvm[layer_name])[ids, :]

# neurons = neural_fea[ids][:, neuron_ids]
neurons = neural_fea

if map_type == 'linreg':
    X = pca_feats
else:
    X = feats
# Y = scale(neurons)
Y = neurons

num_images = X.shape[0]
np.random.seed(0)
# train_ind = np.random.choice(range(num_images), num_images * 9 // 10, replace=False)
train_ind = np.random.choice(range(num_images), num_images // 2, replace=False)
test_ind = np.nonzero(~ np.in1d(range(num_images), train_ind))[0]

all_scores = np.zeros((3, 3))
consistency = np.array(analyst.compute_consistency(neural_reps, 20, population=False)).mean(1)

for ls_ind, ls in enumerate([15, 20, 50]): 
    for ld_ind, ld in enumerate([15, 20, 50]):
        print('Mapping with ls={}, ld={}'.format(ls, ld))
        mapper = SeparableMap(graph=None, max_epochs=700 // 1, tol=0.1, 
                              init_lr=0.01, batch_size=50, ls=ls, ld=ld, num_neurons=neurons.shape[-1],
                              map_type=map_type, inits=None, decay_rate=200 // 1, log_rate=100 // 1
                             )
        mapper.fit(X[train_ind], Y[train_ind])
        preds = mapper.predict(X[test_ind])
        scores = np.array([pearsonr(preds[:, i], Y[test_ind, i])[0] for i in range(preds.shape[-1])])
        all_scores[ls_ind, ld_ind] = np.median(scores)
        
        print('Scores: ', np.median(scores))
        print('Consistency: ', np.median(consistency)) 
        print('Norm. EV: ', np.median(scores ** 2 / consistency ** 2))

print(all_scores)

Mapping with ls=15, ld=15
Initializing...
Epoch: 1, Err Loss: 7725.53, Reg Loss: 23623.11
Epoch: 101, Err Loss: 33.33, Reg Loss: 39.11
Epoch: 201, Err Loss: 29.35, Reg Loss: 36.20
Epoch: 301, Err Loss: 29.91, Reg Loss: 18.33
Epoch: 401, Err Loss: 27.39, Reg Loss: 18.34
Epoch: 501, Err Loss: 26.96, Reg Loss: 17.70
Epoch: 601, Err Loss: 32.76, Reg Loss: 17.71
('Scores: ', 0.4982847943552707)
('Consistency: ', 0.8560523943000633)
('Norm. EV: ', 0.417687094234107)
Mapping with ls=15, ld=20
Initializing...
Epoch: 1, Err Loss: 6248.42, Reg Loss: 23193.62
Epoch: 101, Err Loss: 35.25, Reg Loss: 42.25
Epoch: 201, Err Loss: 25.36, Reg Loss: 39.43
Epoch: 301, Err Loss: 28.74, Reg Loss: 22.45
Epoch: 401, Err Loss: 28.37, Reg Loss: 22.57
Epoch: 501, Err Loss: 26.42, Reg Loss: 22.08
Epoch: 601, Err Loss: 26.88, Reg Loss: 22.12
('Scores: ', 0.49742021156160965)
('Consistency: ', 0.8560523943000633)
('Norm. EV: ', 0.394521432336193)
Mapping with ls=15, ld=50
Initializing...
Epoch: 1, Err Loss: 6237.34

KeyboardInterrupt: 